Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## Define Strategy

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

In [2]:
class SimpleStrategy(Strategy):
    def __init__(self, asset, quantity):
        super().__init__()
        self.asset = asset
        self.quantity = quantity

    def log_all(self, orders, data, ctx, utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(utc)
            self.log_ohlcv(data)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        price = data.get('close')
        quantity = .05

        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, self.asset, quantity, price)
            orders.append(order)
        else:
            order = order_manager.build_market_sell_order(
                ctx.exchange, self.asset, quantity)
            orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)
        print(data.get('utc'))
        self.log_all(orders, data, ctx, data.get('utc'))
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

In [3]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = OHLCVFileFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
asset = Asset(c.ETH, c.BTC)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_backtest'

In [4]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

---------------------------------------
Epoch 1 - Timestep: 2018-01-09T21:30:00
---------------------------------------
OHLCV
    O: 0.0848 | C: 0.0867 | V: 14306.4 | T: 2018-01-09T21:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0867 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 2 - Timestep: 2018-01-09T22:00:00
---------------------------------------
OHLCV
    O: 0.0868 | C: 0.0889 | V: 15933.5 | T: 2018-01-09T22:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0889 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9957 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.99566445, 'used': 0.0, 'total': 0.99566445}
    ETH - {'free': 0.05, 'used': 0.0, 'total': 0.05}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.086711, 'latest_price': 0.0867

2018-01-09 21:30:00
Equal? True
2018-01-09 22:00:00
Equal? True
2018-01-09 22:30:00
Equal? True
2018-01-09 23:00:00
Equal? True
2018-01-09 23:30:00
Equal? True
2018-01-10 00:00:00
Equal? True


---------------------------------------
Epoch 7 - Timestep: 2018-01-10T00:30:00
---------------------------------------
OHLCV
    O: 0.0898 | C: 0.0901 | V: 8079.4 | T: 2018-01-10T00:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0901 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9911 Total Val: 1.0001 PnL: 0.0001 Returns: 0.0001
BALANCE
    BTC - {'free': 0.9911259500000001, 'used': 0.0, 'total': 0.9911259500000001}
    ETH - {'free': 0.10000000000000002, 'used': 0.0, 'total': 0.10000000000000002}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.10000000000000002, 'cost_price': 0.08946933333333332, 'latest_price': 0.089813}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 8 - Timestep: 2018-01-10T01:00:00
---------------------------------------
OHLCV
    O: 0.0901 | C: 0.0907 | V: 9569.9 | T: 2018-01-10T01:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0907 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9866 Total Val: 1.0001 PnL: 0.0001 Returns: 0.

2018-01-10 00:30:00
Equal? True
2018-01-10 01:00:00
Equal? True
2018-01-10 01:30:00
Equal? True
2018-01-10 02:00:00
Equal? True
2018-01-10 02:30:00
Equal? True
2018-01-10 03:00:00
Equal? True
2018-01-10 03:30:00


    BTC - {'free': 0.9822784500000001, 'used': 0.0, 'total': 0.9822784500000001}
    ETH - {'free': 0.2, 'used': 0.0, 'total': 0.2}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.2, 'cost_price': 0.09038402666666667, 'latest_price': 0.09474099999999999}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 14 - Timestep: 2018-01-10T04:00:00
---------------------------------------
OHLCV
    O: 0.0968 | C: 0.0981 | V: 16122.9 | T: 2018-01-10T04:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9774 Total Val: 1.0016 PnL: 0.0011 Returns: 0.0011
BALANCE
    BTC - {'free': 0.9774394500000001, 'used': 0.0, 'total': 0.9774394500000001}
    ETH - {'free': 0.25, 'used': 0.0, 'total': 0.25}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.25, 'cost_price': 0.09166322133333334, 'latest_price': 0.09677999999999999}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 15 - Timestep: 201

...

2018-01-20 04:30:00
Equal? True
2018-01-20 05:00:00
Equal? True
2018-01-20 05:30:00


    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 498 - Timestep: 2018-01-20T06:00:00
---------------------------------------
OHLCV
    O: 0.0884 | C: 0.0890 | V: 6106.9 | T: 2018-01-20T06:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0890 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9914 Total Val: 0.9958 PnL: -0.0042 Returns: -0.0042
BALANCE
    BTC - {'free': 0.9913923999999985, 'used': 0.0, 'total': 0.9913923999999985}
    ETH - {'free': 0.049999999999999906, 'used': 0.0, 'total': 0.049999999999999906}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.049999999999999906, 'cost_price': 0.09069204717682723, 'latest_price': 0.0884}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 499 - Timestep: 2018-01-20T06:30:00
---------------------------------------
OHLCV
    O: 0.0891 | C: 0.0894 | V: 4857.5 | T: 2018-01-20T06:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9869 Tot

Equal? True
2018-01-20 06:00:00
Equal? True
2018-01-20 06:30:00
Equal? True


---------------------------------------
Epoch 500 - Timestep: 2018-01-20T07:00:00
---------------------------------------
OHLCV
    O: 0.0893 | C: 0.0890 | V: 2907.0 | T: 2018-01-20T07:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0890 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9914 Total Val: 0.9959 PnL: -0.0041 Returns: -0.0041
BALANCE
    BTC - {'free': 0.9914120499999985, 'used': 0.0, 'total': 0.9914120499999985}
    ETH - {'free': 0.049999999999999906, 'used': 0.0, 'total': 0.049999999999999906}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.049999999999999906, 'cost_price': 0.0898425235884136, 'latest_price': 0.08938600000000001}
METRICS
    SMA: 5.00
    RSI: 10.00


2018-01-20 07:00:00
Equal? True
No data after prior poll: 2018-01-20 07:00:00


## Simulate Orders

In [ ]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

asset = Asset(c.ETH, c.BTC)
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=[asset],
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_simulate'

In [ ]:
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
exchange = load_exchange(c.BINANCE)
cash = exchange.fetch_balance().get(c.BTC)[BalanceType.FREE]
print(c.BTC, "cash", cash)

In [ ]:
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)

In [ ]:
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
asset = Asset(c.ETH, c.BTC)
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=[asset],
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_live'

In [ ]:
runner.live(experiment_name, exchange, balance, portfolio, feed, strategy)